# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 24 2022 9:18AM,249218,10,CIF10202022,Cosmo International Fragrances,Released
1,Oct 24 2022 9:08AM,249216,10,8687285,"ClearSpec, LLC",Released
2,Oct 24 2022 9:02AM,249214,12,HH-37659,Hush Hush,Released
3,Oct 24 2022 9:02AM,249214,12,HH-37660,Hush Hush,Released
4,Oct 24 2022 9:02AM,249214,12,HH-37661,Hush Hush,Released
5,Oct 24 2022 9:02AM,249214,12,HH-37662,Hush Hush,Released
6,Oct 24 2022 9:02AM,249214,12,HH-37664,Hush Hush,Released
7,Oct 24 2022 9:02AM,249214,12,HH-37665,Hush Hush,Released
8,Oct 24 2022 9:02AM,249214,12,HH-37666,Hush Hush,Released
9,Oct 24 2022 9:02AM,249214,12,HH-37667,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
11,249214,Released,65
12,249215,Executing,1
13,249215,Released,1
14,249216,Released,1
15,249218,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249213,NaN,1.0
249214,NaN,65.0
249215,1.0,1.0
249216,NaN,1.0
249218,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249196,0.0,12.0
249204,0.0,10.0
249205,0.0,3.0
249206,1.0,11.0
249207,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249196,0,12
249204,0,10
249205,0,3
249206,1,11
249207,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249196,0,12
1,249204,0,10
2,249205,0,3
3,249206,1,11
4,249207,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249196,,12
1,249204,,10
2,249205,,3
3,249206,1,11
4,249207,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 24 2022 9:18AM,249218,10,Cosmo International Fragrances
1,Oct 24 2022 9:08AM,249216,10,"ClearSpec, LLC"
2,Oct 24 2022 9:02AM,249214,12,Hush Hush
67,Oct 24 2022 9:00AM,249215,12,Hush Hush
69,Oct 24 2022 8:53AM,249211,10,"Methapharm, Inc."
94,Oct 24 2022 8:51AM,249213,10,"Virtue Rx, LLC"
95,Oct 24 2022 8:50AM,249212,10,"Methapharm, Inc."
104,Oct 24 2022 8:50AM,249212,10,Methapharm-G
105,Oct 24 2022 8:37AM,249209,10,ISDIN Corporation
124,Oct 24 2022 8:30AM,249196,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 24 2022 9:18AM,249218,10,Cosmo International Fragrances,,1
1,Oct 24 2022 9:08AM,249216,10,"ClearSpec, LLC",,1
2,Oct 24 2022 9:02AM,249214,12,Hush Hush,,65
3,Oct 24 2022 9:00AM,249215,12,Hush Hush,1,1
4,Oct 24 2022 8:53AM,249211,10,"Methapharm, Inc.",,25
5,Oct 24 2022 8:51AM,249213,10,"Virtue Rx, LLC",,1
6,Oct 24 2022 8:50AM,249212,10,"Methapharm, Inc.",,10
7,Oct 24 2022 8:50AM,249212,10,Methapharm-G,,10
8,Oct 24 2022 8:37AM,249209,10,ISDIN Corporation,1,18
9,Oct 24 2022 8:30AM,249196,10,ISDIN Corporation,,12


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 24 2022 9:18AM,249218,10,Cosmo International Fragrances,1,
1,Oct 24 2022 9:08AM,249216,10,"ClearSpec, LLC",1,
2,Oct 24 2022 9:02AM,249214,12,Hush Hush,65,
3,Oct 24 2022 9:00AM,249215,12,Hush Hush,1,1
4,Oct 24 2022 8:53AM,249211,10,"Methapharm, Inc.",25,
5,Oct 24 2022 8:51AM,249213,10,"Virtue Rx, LLC",1,
6,Oct 24 2022 8:50AM,249212,10,"Methapharm, Inc.",10,
7,Oct 24 2022 8:50AM,249212,10,Methapharm-G,10,
8,Oct 24 2022 8:37AM,249209,10,ISDIN Corporation,18,1
9,Oct 24 2022 8:30AM,249196,10,ISDIN Corporation,12,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 24 2022 9:18AM,249218,10,Cosmo International Fragrances,1,
1,Oct 24 2022 9:08AM,249216,10,"ClearSpec, LLC",1,
2,Oct 24 2022 9:02AM,249214,12,Hush Hush,65,
3,Oct 24 2022 9:00AM,249215,12,Hush Hush,1,1
4,Oct 24 2022 8:53AM,249211,10,"Methapharm, Inc.",25,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 24 2022 9:18AM,249218,10,Cosmo International Fragrances,1,NaN
1,Oct 24 2022 9:08AM,249216,10,"ClearSpec, LLC",1,NaN
2,Oct 24 2022 9:02AM,249214,12,Hush Hush,65,NaN
3,Oct 24 2022 9:00AM,249215,12,Hush Hush,1,1.0
4,Oct 24 2022 8:53AM,249211,10,"Methapharm, Inc.",25,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 24 2022 9:18AM,249218,10,Cosmo International Fragrances,1,0.0
1,Oct 24 2022 9:08AM,249216,10,"ClearSpec, LLC",1,0.0
2,Oct 24 2022 9:02AM,249214,12,Hush Hush,65,0.0
3,Oct 24 2022 9:00AM,249215,12,Hush Hush,1,1.0
4,Oct 24 2022 8:53AM,249211,10,"Methapharm, Inc.",25,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1993687,78,1.0
12,498429,66,1.0
19,747615,24,1.0
20,249207,2,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1993687,78,1.0
1,12,498429,66,1.0
2,19,747615,24,1.0
3,20,249207,2,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,78,1.0
1,12,66,1.0
2,19,24,1.0
3,20,2,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,78.0
1,12,Released,66.0
2,19,Released,24.0
3,20,Released,2.0
4,10,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20
Status,,,,
Executing,1.0,1.0,1.0,0.0
Released,78.0,66.0,24.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20
0,Executing,1.0,1.0,1.0,0.0
1,Released,78.0,66.0,24.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20
0,Executing,1.0,1.0,1.0,0.0
1,Released,78.0,66.0,24.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()